In [ ]:
# talib のインストールが難しく Docker 上でしか動かせられない
# 2023/2/27 変更、 Docker の起動ができず、デバッグできていない

In [ ]:
import pandas as pd
import numpy as np
import talib as ta

import warnings
warnings.simplefilter('ignore')

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

In [ ]:
nk400 = pd.read_csv('../stock_data/nk400.csv', index_col='Date')
nk200 = pd.read_csv('../stock_data/nk200.csv', index_col='Date')

# index を datetime に
nk400.index = pd.to_datetime(nk400.index)
nk200.index = pd.to_datetime(nk200.index)

In [ ]:
# CSVを読み込み、指数構成銘柄のリストを作成
nk400_code = pd.read_csv('../constituent_stocks/jpx_nikkei_index_400_weight_jp.csv', encoding='shift-jis')['コード'][:-1].tolist()
nk200_code = pd.read_csv('../constituent_stocks/jpx_nikkei_mid_small_weight_jp.csv', encoding='shift-jis')['コード'][:-1].tolist()

In [ ]:
# 基準は timeperiod=20, nbdevup=2, nbdevdn=2, matype=0
param_list = [[20, 2, 2, 0], [20, 1, 1, 0], [20, 2, 2, 1], [20, 3, 3, 0]]

In [ ]:
# 個別銘柄のボリンジャーバンドの作成、個別・市場リターンの計算
# 週ごとについても行う
# DataFrameを返す

# timeperiod 計算期間
# nbdevup、nbdevdn 上側、下側バンドの σ
# matype 0 単純移動平均線   1 指数平滑移動平均線   2 加重移動平均線

def company_stock(code, index_df):
    df_d = index_df[['Index', code]].dropna()
    
    price=df_d[code]
    index_=df_d['Index']
    # ボリンジャーバンドの作成
    df_d['upper'], df_d['middle'], df_d['lower'] = ta.BBANDS(price, timeperiod=timeperiod, nbdevup=nbdevup, nbdevdn=nbdevdn, matype=matype)
    # 個別リターン
    df_d['r_i'] = (price-price.shift())/price.shift()
    # 市場リターン
    df_d['r_m'] = (index_-index_.shift())/index_.shift()

    # 週ごとにも集計を行う
    df_w = df_d.resample('W', loffset=pd.offsets.timedelta()).agg({'close': 'last'}).droplevel(0, axis=1)
    # 2019年、改元の際、１日も取引がない週があったのでそれを除く（除かないとボリンジャーバンドが正しく出ない）
    df_w = df_w.dropna()

    # 週ごとも同様に
    price=df_w[code]
    index_=df_w['Index']
    # ボリンジャーバンドの作成
    df_w['upper'], df_w['middle'], df_w['lower'] = ta.BBANDS(price, timeperiod=timeperiod, nbdevup=nbdevup, nbdevdn=nbdevdn, matype=matype)
    # 個別リターン
    df_w['r_i'] = (price-price.shift())/price.shift()
    # 市場リターン
    df_w['r_m'] = (index_-index_.shift())/index_.shift()
    
    return df_d, df_w

In [ ]:
# イベントウィンドウ：イベント日後５日（週）
# 推定期間：イベント日を t=0 とし、-155≦ t ≦-6 の150営業日（週）
# 推定期間が足りない場合は除外する

In [ ]:
# 特定銘柄について、全期間のDataFrameとイベント日を渡すとイベント日後５日の ARの list を返す関数
def get_ar(df_all, event_date):
    #df_all = df_all
    # イベント日をもとにtを決定する
    t_list = list(range(-len(df_all.loc[:event_date])+1, 0)) + list(range(0, len(df_all.loc[event_date:])))
    df_all['t'] = t_list

    # tの最小が-156以下（推定期間が十分）であり (最小が-155だと最初はリターンが NaNになるので不適)
    # tの最大値が 5以上（イベントウィンドウが十分）である ならば、イベントスタディを行う
    if (df_all.iloc[0]['t'] <= -156) & (df_all.iloc[-1]['t']>=5):

        # 推定期間からマーケットモデルを作成
        estimation = df_all[(-155<=df_all['t']) & (df_all['t']<=-6)]

        x = estimation["r_m"]
        y = estimation["r_i"]

        #単回帰分析モデルの作成
        beta, alpha = np.polyfit(x, y, 1)

        # ベンチマーク
        df_all['r_e'] = alpha + beta*df_all['r_m']

        # ARの計算
        df_all['ar'] = df_all['r_i']-df_all['r_e']

        # 1≦t≦5 におけるAR
        event_window_df = df_all[(1<=df_all['t']) & (df_all['t']<=5)]
        ar_list = list(event_window_df['ar'])
        
    else:
        ar_list = [None]*5
        
    return ar_list

In [ ]:
# 結果をリストに格納する関数
def store_result(upper_or_lower, data_list, df_all):
    for event_date in upper_or_lower.index:
        data_list.append([code, event_date] + get_ar(df_all, event_date))

In [ ]:
# listをDataFrameにまとめてCSVに書き出す関数
def to_dataframe(data, csv_name):
    data = pd.DataFrame(data, columns=['code', 'date', 'ar1', 'ar2', 'ar3', 'ar4', 'ar5'])

    # CAR算出
    data['car1'] = data['ar1']
    data['car2'] = data['car1'] + data['ar2']
    data['car3'] = data['car2'] + data['ar3']
    data['car4'] = data['car3'] + data['ar4']
    data['car5'] = data['car4'] + data['ar5']

    data.to_csv(f'../output/{timeperiod}_{nbdevup}_{nbdevdn}_{matype}/{csv_name}.csv', encoding='shift-jis')

In [ ]:
def event_study(param):
    timeperiod, nbdevup, nbdevdn, matype = param

    # 格納するリスト
    nk400_d_upper = []
    nk400_d_lower = []
    nk400_w_upper = []
    nk400_w_lower = []

    for code in nk400_code:
        code = str(int(code))

        df_d, df_w = company_stock(code, nk400)

        d_upper = df_d[df_d[code]>df_d['upper']]
        d_lower = df_d[df_d[code]<df_d['lower']]

        w_upper = df_w[df_w[code]>df_w['upper']]
        w_lower = df_w[df_w[code]<df_w['lower']]

        store_result(d_upper, nk400_d_upper, df_d)
        store_result(d_lower, nk400_d_lower, df_d)
        store_result(w_upper, nk400_w_upper, df_w)
        store_result(w_lower, nk400_w_lower, df_w)

    # 格納するリスト
    nk200_d_upper = []
    nk200_d_lower = []
    nk200_w_upper = []
    nk200_w_lower = []

    for code in nk200_code:
        code = str(int(code))

        df_d, df_w = company_stock(code, nk200)

        d_upper = df_d[df_d[code]>df_d['upper']]
        d_lower = df_d[df_d[code]<df_d['lower']]

        w_upper = df_w[df_w[code]>df_w['upper']]
        w_lower = df_w[df_w[code]<df_w['lower']]

        store_result(d_upper, nk200_d_upper, df_d)
        store_result(d_lower, nk200_d_lower, df_d)
        store_result(w_upper, nk200_w_upper, df_w)
        store_result(w_lower, nk200_w_lower, df_w)

    data_list = [nk400_d_upper, nk400_d_lower, nk400_w_upper, nk400_w_lower,
                  nk200_d_upper, nk200_d_lower, nk200_w_upper, nk200_w_lower]
    name_list = ['nk400_d_upper', 'nk400_d_lower', 'nk400_w_upper', 'nk400_w_lower',
                  'nk200_d_upper', 'nk200_d_lower', 'nk200_w_upper', 'nk200_w_lower']
    for i in range(0, len(data_list)):
        to_dataframe(data_list[i], name_list[i])

In [ ]:
# 各パラメータについてイベントスタディを実行
for param in param_list:
    event_study(param)